# Something a little harder - buffer  & spatial join WDPA and GHSL

In [ ]:
import geopandas as gpd
import time, os, sys
gpd.options.use_pygeos=True

In [ ]:
import pygeos

In [ ]:
# from geopandas_parallelise import paralellise
# parallelise(gpd)
# gpd.sjoin(left,right,op, workers=XX)
# gdf.buffer_utm(XXX, workers=XX, unit='m')

In [ ]:
# set root path
root = os.path.abspath(os.path.join(os.getcwd(),'..'))
os.environ['PYTHONPATH'] = root
sys.path.append(root)

## Load the Data

In [ ]:
### get some data

In [ ]:
!wget -O $PYTHONPATH/data/wdpa.zip https://d1gam3xoknrgr2.cloudfront.net/current/WDPA_Feb2021_Public_shp.zip

In [ ]:
!wget -O $PYTHONPATH/data/ghsl.zip http://cidportal.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_STAT_UCDB2015MT_GLOBE_R2019A/V1-2/GHS_STAT_UCDB2015MT_GLOBE_R2019A.zip

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile(os.path.join(root,'data','wdpa.zip'), 'r') as z:
    z.extractall(os.path.join(root,'data'))

In [ ]:
wdpa = gpd.read_file('zip://'+os.path.join(root,'data','WDPA_Feb2021_Public_shp_0.zip!WDPA_Feb2021_Public_shp-polygons.shp'))

In [ ]:
ghsl = gpd.read_file('zip://'+os.path.join(root,'data','ghsl.zip!GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg'))

## Plot the data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(24,16))
wdpa.plot(ax=ax,color='g')
ghsl.plot(ax=ax,color='r')
plt.show()

## Do a buffered intersect

In [ ]:
from geopandas_parallelise.buffer import *

In [ ]:
buffer_dist=10000
n_workers=4

In [ ]:
start_time = time.time()
ghsl['buffer_geom'] = mp_buffer(ghsl,buffer_dist,n_workers)
print (f'elapsed_time {time.time()-start_time}s')

In [ ]:
ghsl = ghsl.set_geometry('buffer_geom')

In [ ]:
# Plot again
fig,ax = plt.subplots(1,1,figsize=(24,16))
wdpa.plot(ax=ax,color='g')
ghsl.plot(ax=ax,color='r')
plt.show()

## Do a spatial join with buffered data

In [ ]:
from geopandas_parallelise.sjoin import *

In [ ]:
# spatial join
start_time = time.time()
joined = mp_sjoin(ghsl,wdpa,n_workers=4,how='inner',op='intersects')
print (f'Spatial joined time {time.time()-start_time}')